In [1]:
import pandas as pd
import numpy as np
from numpy import sort
import copy
import math
import cmath
import datetime

from sklearn.feature_selection import RFE,RFECV
from sklearn.feature_selection import SelectFromModel
import joblib

from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

import matplotlib.pyplot as plt

# Function

In [2]:
def calculate_LR(df1,df2):
    """
    df1:cv training set
    df2:cv test set
    """
    # log ((2+c)/(1+c)) + log ((2+c)/ (1+c)), {c==1}

    p = {}
    n = {}
    for index,row in df1.iterrows():
        if(pd.isna(row['ancestor'])):
            continue
        for i in row['ancestor'].split(','):
            if i not in p.keys():
                p[i]=1
                n[i]=1
            if(row['is_del']==1):
                p[i]+=1
            else:
                n[i]+=1
                
    l = copy.deepcopy(p)
    for i in l.keys():
        l[i]=math.log(p[i]/n[i]) 
    l
    
    def LR_add(x):
        sum=0
        if(pd.isna(x)):
            return sum
        for i in x.split(','):
            if i in l:
                sum = sum + l[i]
        return sum
    df1['LR'] = df1['ancestor'].apply(lambda x:LR_add(x))
    df2['LR'] = df2['ancestor'].apply(lambda x:LR_add(x))
    df1 = df1.drop(columns=['ancestor'])
    df2 = df2.drop(columns=['ancestor'])
    return df1,df2

In [3]:

def calculate_PA(df1,df2):
    """
    df1:cv training set
    df2:cv test set
    """
    # log ((2+c)/(1+c)) + log ((2+c)/ (1+c)), {c==1}


    p = {}
    n = {}
    for index,row in df1.iterrows():
        if(pd.isna(row['site'])):
            continue
        for i in row['site'].split(','):
            if i!='':
                if i not in p.keys():
                    p[i]=1
                    n[i]=1
                if(row['is_del']==1):
                    p[i]+=1
                else:
                    n[i]+=1
                
    s = copy.deepcopy(p)
    for i in s.keys():
        s[i]=math.log(p[i]/n[i]) 
    s
    

    def PA_add(x):
        sum=0
        if(pd.isna(x)):
            return sum
        for i in x.split(','):
            if i != '' and i in s:
                sum = sum + s[i]
        return sum
    df1['PA'] = df1['site'].apply(lambda x:PA_add(x))
    df2['PA'] = df2['site'].apply(lambda x:PA_add(x))
    df1 = df1.drop(columns=['site'])
    df2 = df2.drop(columns=['site'])
    return df1,df2

# Bootstrap抽样

# CV

### All_species

In [7]:
for i in range(1,11):
    df1 = pd.read_csv('./CV_dataset/All_species/All_species_train{}.csv'.format(i))
    df2 = pd.read_csv('./CV_dataset/All_species/All_species_validation{}.csv'.format(i))
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)
    for j in range(200):
        train = df1.sample(frac=1.0,replace=True)
        train.to_csv("All_species_bootstraps/bootstrap{}/bootstrap_Combine_train_{}.csv".format(i,j),float_format='%.3f',index=None)

### Human

In [ ]:
for i in range(1,11):
    df1 = pd.read_csv('./CV_dataset/Human/Human_train{}.csv'.format(i))
    df2 = pd.read_csv('./CV_dataset/Human/Human_validation{}.csv'.format(i))
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)
    for j in range(200):
        train = df1.sample(frac=1.0,replace=True)
        train.to_csv("Human_bootstraps/bootstrap{}/bootstrap_Combine_train_{}.csv".format(i,j),float_format='%.3f',index=None)

### Animal

In [4]:
for i in range(1,11):
    df1 = pd.read_csv('./CV_dataset/Animal/Animal_train{}.csv'.format(i))
    df2 = pd.read_csv('./CV_dataset/Animal/Animal_validation{}.csv'.format(i))
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)
    for j in range(200):
        train = df1.sample(frac=1.0,replace=True)
        train.to_csv("Animal_bootstraps/bootstrap{}/bootstrap_Combine_train_{}.csv".format(i,j),float_format='%.3f',index=None)

### Plant

In [5]:
for i in range(1,11):
    df1 = pd.read_csv('./CV_dataset/Plant/Plant_train{}.csv'.format(i))
    df2 = pd.read_csv('./CV_dataset/Plant/Plant_validation{}.csv'.format(i))
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)
    for j in range(200):
        train = df1.sample(frac=1.0,replace=True)
        train.to_csv("Plant_bootstraps/bootstrap{}/bootstrap_Combine_train_{}.csv".format(i,j),float_format='%.3f',index=None)

# Blind

In [ ]:
def sample_blind(species): 
    
    data1 = pd.read_csv('./CV_dataset/{}/{}_train1.csv'.format(species,species))
    data2 = pd.read_csv('./CV_dataset/{}/{}_validation1.csv'.format(species,species))
    df1 = pd.concat([data1,data2], ignore_index=True)
    df2 = pd.read_csv('Dataset/Blind_{}.csv'.format(species))
    
    df1,df2 = calculate_LR(df1,df2)
    df1,df2 = calculate_PA(df1,df2)

    for i in range(200):
        train = df1.sample(frac=1.0,replace=True)
        train.to_csv("Blind/{}/bootstrap_Combine_train_{}.csv".format(species,i),float_format='%.3f',index=None)

In [ ]:
sample_blind('All_species')
sample_blind('Human')
sample_blind('Animal')
sample_blind('Plant')